# Modelo de recomendación - Item a Item

Importamos las librerías y el dataset

In [121]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

Decidí utilizar el dataset games pues contiene los datos más relevantes para el aprendizaje del modelo. Es una muestra del 12% del dataframe que asegura el deployment y adicionalmente elimina los géneros que no tenían relevancia importante de acuerdo a su cantidad sobre el total del dataframe.

In [122]:
df = pd.read_csv(r'df_modelo_recomendacion.csv')

In [123]:
df.genres.value_counts()

genres
Indie         1500
Action        1096
Adventure      857
Casual         756
Strategy       646
Simulation     608
RPG            451
Name: count, dtype: int64

Definimos el vector y las variables para asignar la recomendación en la matriz

In [124]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df["publisher"] + " " + df["genres"] + " " + df["app_name"])

Calculo la matriz de similitud de coseno basada en la matriz TF-IDF:

In [125]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

Definimos una función que nos garantice:

1. Que la recomendación no arrojara el mismo resultado del ítem que se selecciona como base para la recomendación.
2. Lista que filtre los repetidos, para que las 5 sugerencias sean positivas.


In [126]:
def recomendaciones(id):
    item_indice = df[df["id"] == id].index[0] # Obtenemos el índice del item input en nuestro DataFrame
    items_similares = list(enumerate(cosine_sim[item_indice])) # Generamos lista con los items similares que devolvió el modelo
    similar_items_organizados = sorted(items_similares, key=lambda x: x[1], reverse=True) # Organizamos los items del punto anterior
    indices = [index for index, _ in similar_items_organizados[1:10]] # Recorremos los ítems para traer las 5 recomendaciones

    items_recomendados = df.iloc[indices]["id"].tolist()

    # Obtengo los nombres de los ítems recomendados para el retorno de la función, no permito repetidos y limito a 5 el tamaño de la lista
    lista_nombres = [df[df['id'] == elemento]['app_name'].iloc[0] for elemento in items_recomendados if elemento != id]
    lista_nombres = lista_nombres[:5]

    diccionario = {f'Juego {i+1}': nombre for i, nombre in enumerate(lista_nombres)}

    return diccionario

recomendaciones(610660)

{'Juego 1': 'The guard of dungeon',
 'Juego 2': '3D Hardcore Cube 2',
 'Juego 3': 'Roads of Rome 3',
 'Juego 4': 'Slash it 2 - Russian Edition Pack',
 'Juego 5': 'Animals Memory'}